In [1]:
import json
import sqlite3
import requests
import os
from kaggle import KaggleApi
import pandas as pd
import ast

api = KaggleApi()

api.authenticate()


# Download Data

In [2]:
def create_file_directory(directory_path):
    """Creates a file directory.

    Args:
      directory_path: The path to the directory to create.
    """

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)


def download_github_json_file(repo_url, file_path):
    """Downloads a JSON file from a GitHub repo.

    Args:
      repo_url: The URL of the GitHub repo.
      file_path: The path to the JSON file in the repo.
    """

    response = requests.get(repo_url + '/raw/main/' + file_path)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
    else:
        raise Exception(
            'Failed to download JSON file: {}'.format(response.status_code))


In [3]:
# create_file_directory("Data/Bronze/manami-project/")
# create_file_directory(
#     "Data/Bronze/Kaggle/myanimelist-dataset-animes-profiles-reviews/")
# create_file_directory("Data/Bronze/Kaggle/anime-recommendations-database/")
# create_file_directory("Data/Silver/")


In [4]:
# download_github_json_file(
#     "https://github.com/manami-project/anime-offline-database/blob/master/anime-offline-database.json?raw=true",
#     "Data/Bronze/manami-project/anime-offline-database.json")


In [5]:
# api.dataset_download_files(dataset="CooperUnion/anime-recommendations-database",
#                            path="Data/Bronze/Kaggle/anime-recommendations-database", unzip=True)

# api.dataset_download_files(dataset="marlesson/myanimelist-dataset-animes-profiles-reviews",
#                            path="Data/Bronze/Kaggle/myanimelist-dataset-animes-profiles-reviews", unzip=True)


# Merge Data

In [6]:
with open('Data/Bronze/manami-project/anime-offline-database.json', 'r') as json_file:
    data_json = json.load(json_file)

git_anime = pd.DataFrame(data_json['data'])

kaggle_anime = pd.read_csv(
    "Data/Bronze/Kaggle/myanimelist-dataset-animes-profiles-reviews/animes.csv")

kaggle_to_merge = kaggle_anime[[
    'title', 'members',	'popularity',	'ranked',	'score']]

data_full = pd.merge(git_anime, kaggle_to_merge, how='left', on='title')

data_full.to_json('Data/Silver/anime-full.json', orient='records')


# Create SQLITE DB

In [10]:
conn = sqlite3.connect('anime-test1.db')
cursor = conn.cursor()

script = open('Create Tables.sql', 'r').read()
cursor.executescript(script)


# Load Anime Data

In [13]:
with open('Data/Silver/anime-full.json', 'r') as json_file:
    anime_list = json.load(json_file)


# for data in anime_list:
#     cursor.execute('''
#         INSERT INTO anime (title, type, episodes, year, season, status, score, ranking, popularity, members)
#         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#     ''', (data['title'], data['type'], data['episodes'], data['animeSeason']['year'], data['animeSeason']['season'], data['status'], data['score'], data['ranked'], data['popularity'], data['members']))

#     anime_id = cursor.lastrowid

#     for synonym in data['synonyms']:
#         cursor.execute(
#             'INSERT INTO synonyms (synonym, anime_id) VALUES (?, ?)', (synonym, anime_id))

#     for source in data['sources']:
#         cursor.execute('INSERT INTO source (name) VALUES (?)', (source,))
#         source_id = cursor.lastrowid  # Get the source_id of the inserted source

#         cursor.execute(
#             'SELECT 1 FROM source_bridging WHERE anime_id = ? AND source_id = ?', (anime_id, source_id))
#         existing_relationship = cursor.fetchone()

#         if not existing_relationship:
#             cursor.execute(
#                 'INSERT INTO source_bridging (anime_id, source_id) VALUES (?, ?)', (anime_id, source_id))

#     for tag in data['tags']:
#         cursor.execute('INSERT OR IGNORE INTO tags (tag) VALUES (?)', (tag,))
#         cursor.execute('SELECT tag_id FROM tags WHERE tag=?', (tag,))
#         tag_id = cursor.fetchone()[0]
#         cursor.execute(
#             'INSERT INTO tags_bridging (anime_id, tag_id) VALUES (?, ?)', (anime_id, tag_id))

for data in anime_list:
    cursor.execute('''
        INSERT INTO anime (title, type, episodes, year, season, status, score, ranking, popularity, members)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (data['title'], data['type'], data['episodes'], data['animeSeason']['year'], data['animeSeason']['season'], data['status'], data['score'], data['ranked'], data['popularity'], data['members']))

    anime_id = cursor.lastrowid

    for synonym in data['synonyms']:
        cursor.execute(
            'INSERT INTO synonyms (synonym, anime_id) VALUES (?, ?)', (synonym, anime_id))
        
    for source in data['sources']:
        cursor.execute(
            'INSERT OR IGNORE INTO source (name) VALUES (?)', (source,))
        cursor.execute(
            'SELECT source_id FROM source WHERE name = ?', (source,))
        source_row = cursor.fetchone()

        if source_row:
            source_id = source_row[0]
        else:
            source_id = cursor.lastrowid

        cursor.execute(
            'INSERT OR IGNORE INTO source_bridging (anime_id, source_id) VALUES (?, ?)', (anime_id, source_id))

    for tag in data['tags']:
        cursor.execute(
            'INSERT OR IGNORE INTO tags (tag) VALUES (?)', (tag,))
        cursor.execute(
            'SELECT tag_id FROM tags WHERE tag = ?', (tag,))
        tag_row = cursor.fetchone()

        if tag_row:
            tag_id = tag_row[0]
        else:
            tag_id = cursor.lastrowid

        cursor.execute(
            'INSERT OR IGNORE INTO tags_bridging (anime_id, tag_id) VALUES (?, ?)', (anime_id, tag_id))

# Load Users Data 

In [14]:
profiles = pd.read_csv("Data/Bronze/Kaggle/myanimelist-dataset-animes-profiles-reviews/profiles.csv")
profiles.rename(columns={'profile': 'username'}, inplace=True)
profile_to_add = profiles[[
    'username',
    'gender',
    'birthday',
    'link'
]]

profile_to_add.to_csv('Data/Silver/profiles.csv', index=False)

profile_to_add.to_sql('users', conn, if_exists='append', index=False)

81727

# Load Reviews Data 

In [15]:
reviews = pd.read_csv(
    "Data/Bronze/Kaggle/myanimelist-dataset-animes-profiles-reviews/reviews.csv")

reviews['scores'] = reviews['scores'].apply(ast.literal_eval)

reviews = pd.concat([reviews.drop(['scores', 'score'], axis=1),
                    reviews['scores'].apply(pd.Series)], axis=1)

reviews.rename(columns={
    'Overall': 'overall_score',
    'Story': 'story_score',
    'Animation': 'animation_score',
    'Sound': 'sound_score',
    'Character': 'character_score',
    'Enjoyment': 'enjoyment_score',
    'text': 'review',
    'profile': 'username',
    'anime_uid': 'anime_id'
}, inplace=True)

kaggle_anime = kaggle_anime[['uid', 'title']]

kaggle_anime.rename(columns={'uid': 'anime_id'}, inplace=True)

reviews = pd.merge(reviews, kaggle_anime, how='left', on='anime_id')

anime_table = cursor.execute('''SELECT*FROM anime''')

anime_table = pd.DataFrame(anime_table.fetchall())

anime_table.columns = [x[0] for x in cursor.description]

anime_table_to_merge = anime_table[['anime_id', 'title']]

In [16]:
reviews = pd.concat([reviews, anime_table_to_merge], axis=1)

users = cursor.execute('''SELECT*FROM users''')
users = pd.DataFrame(users.fetchall())
users.columns = [x[0] for x in cursor.description]
users = users[['username', 'user_id']]
reviews = pd.merge(reviews, users, how='left', on='username')

In [17]:
reviews = reviews[[
    'review',
    'overall_score', 
    'story_score',
    'animation_score',
    'sound_score',
    'character_score',
    'enjoyment_score',
    'link',
    'anime_id',
    'user_id'
]]

reviews.to_csv('Data/Silver/reviews.csv', index=False)

reviews.to_sql('reviews', conn, if_exists='append', index=False)

811636

# Load user_favorite_anime Data

In [18]:
rev = cursor.execute('''SELECT*FROM reviews''')

rev = pd.DataFrame(rev.fetchall())

rev.columns = [x[0] for x in cursor.description]

profiles.rename(columns={'favorites_anime': 'anime_id'}, inplace=True)


profiles['anime_id'] = profiles['anime_id'].apply(
    pd.to_numeric, args=('coerce',))

user_favorite_anime = pd.merge(
    profiles, kaggle_anime, how='left', on='anime_id')

user_favorite_anime = user_favorite_anime[['username', 'title']]

anime_ids = pd.read_sql_query('''SELECT anime_id, title FROM anime''', conn)

anime_ids = pd.DataFrame(anime_ids)

user_ids = pd.read_sql_query('''SELECT user_id, username FROM users''', conn)

user_ids = pd.DataFrame(user_ids)

user_favorite_anime = pd.merge(
    user_favorite_anime, anime_ids, how='left', on='title')

user_favorite_anime = pd.merge(
    user_favorite_anime, user_ids, how='left', on='username')

user_favorite_anime = user_favorite_anime[['user_id', 'anime_id']]

user_favorite_anime.drop_duplicates(
    subset=['user_id', 'anime_id'], inplace=True)

user_favorite_anime.to_csv('Data/Silver/user_favorite_anime.csv', index=False)

user_favorite_anime.to_sql('user_favorite_anime',
                           conn, if_exists='append', index=False)


81727

In [19]:
conn.commit()
conn.close()